<a href="https://colab.research.google.com/github/colinmcnamara/austin_langchain/blob/main/labs/LangChain_106/langchain_runnablepassthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# RunnableParallel, RunnablePassthrough, and itemgetter in Action

1. [RunnableParallel](https://python.langchain.com/docs/expression_language/primitives/parallel/#parallelize-steps) lets you execute runnables in parallel.
2. [RunnablePassthrough](https://python.langchain.com/docs/expression_language/primitives/passthrough/) lets you pass parameters through from one step to the next in a chain.
3. [itemgetter](https://docs.python.org/3/library/operator.html#operator.itemgetter) returns a function that lets you extract a specific from a collection, dictionary, etc.

You can use these together to create complex chains by composing simple chains.

In the below example, we will construct two simple chains:

1. `tweet_generator` takes prompt from a user and generates a tweet
2. `tweet_fixer` takes a tweet and a mood from the user and generates a new tweet to match the mood

We will then create a new chain `tweet_chain` that combines both of these chains into a single chain.


## Imports


In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

## Prompt


In [2]:
prompt = "Langchain releases AI middleware to create applications using LLMs. https://langchain.com"

## LLM


In [3]:
llm = ChatOllama(model="mistral")

## `tweet_generator` chain


In [4]:
first_prompt = PromptTemplate.from_template(
    """
You are an expert AI tweet generator.
Observe the user's prompt, and generate a witty tweet, and include emojis and hashtags.

Prompt: {prompt}
Tweet: """
)
tweet_generator = first_prompt | llm | StrOutputParser()

## Running `tweet_generator`


In [5]:
tweet = tweet_generator.invoke({"prompt": prompt})
tweet

' "🚀 Exciting news! Langchain unveils its new AI middleware for developing apps with Language Models 💻 #LLMtech #AIappdev. Say goodbye to coding headaches and hello to smarter, more efficient applications. Let\'s build the future together! 🤝 #innovation #technology"'

## `tweet_fixer` chain


In [6]:
second_prompt = PromptTemplate.from_template(
    """
You are an expert AI tweet fixer.
Fix user's original tweet and based on the mood.

Original Tweet: {tweet}
Mood: {mood}
Fixed Tweet: """
)

tweet_fixer = second_prompt | llm | StrOutputParser()

## Running `tweet_fixer` with tweet from previous response


In [7]:
mood = "funny"

In [8]:
fixed_tweet = tweet_fixer.invoke(
    {
        "tweet": tweet,
        "mood": mood,
    }
)
fixed_tweet

" 🚀 Fun fact: Langchain's new AI middleware for Language Model apps is so smart, it might even start coding for us next! 💻 #LLMtech #AIappdev #CodingMonkeyRetired. Goodbye spaghetti code, hello sushi roll applications! Let's build a future where AI does the heavy lifting – or at least the coding! 😂 #innovation #technology"

## Combining `tweet_generator` and `tweet_fixer` into `tweet_chain`


In [9]:
tweet_chain = (
    RunnableParallel(
        {"mood": RunnablePassthrough() | itemgetter("mood"), "tweet": tweet_generator}
    )
    | tweet_fixer
)

## Running `tweet_chain`


In [10]:
mood = "sarcastic"

In [11]:
tweet_chain.invoke({"prompt": prompt, "mood": mood})

" 😐 Oh joy, another middleware for developers to deal with. This time it's Langchain's turn with their AI offering for LLMs. Can't wait to navigate yet another integration process. #Sarcasm #AI #MachineLearning #AppDev #Langchain #TechInnovation 💻🚀 #LanguageModels #CodeComplication"